In [ ]:
## importamos las librerías necesarias
import requests ## permite realizar solicitudes HTTP
from bs4 import BeautifulSoup ## crea la "sopa", es decir, extrae el código de HTML de la página para poder obtener los datos 
import re ## busca patrones de texto
import pandas as pd ##crea y manipula Data Frames

In [ ]:
### El siguiente código accede a la página de Transfermarkt que tiene el ranking FIFA del 9 de enero del 2023.
### Después, descarga los enlaces para el perfil de cada equipo y los modifica para poderlos utilizar en el siguiente código.
## Para finalizar, guarda los enlaces modificados, el nombre de la selección, el año de análisis y un rango de fecha de un año
## específico, esto para utilizarlo en el código siguiente para generar los enlaces a la página de rendimiento en el equipo de 
## cada jugador convocado.

## Nota: para detalles de cada línea de código revisar el archivo "Informacion_JugadoresConvocados.ipynb"

page= 'https://www.transfermarkt.mx/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2023-01-09'
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
page_tree = requests.get(page, headers=headers)
soup = BeautifulSoup(page_tree.content, 'html.parser')
table= soup.find('table', {'class':'items'})
table_body= table.find('tbody')
tds= table_body.find_all('td', {'class':'hauptlink'})
seleccionlink={'Selección':[],'Link':[],'IdSelección':[],'Year':[],'Inicio':[],'Fin':[],'Inicio (T-1)':[],'Fin (T-1)':[]}
linksolo=[]
tdsnecesarios=[]
seleccion=[]
for y in range(0, len(tds), 2):
    tdsnecesarios.append(y)
for j, te in enumerate(tds):
    for sele in range(0, len(tdsnecesarios)):
        if j==tdsnecesarios[sele]:
            img=te.find('img')
            a=te.find('a')
            seleccion.append(img['alt'])
            linksolo.append(a['href'])
for l in range(0,len(seleccion)):
    for year in range(2009, 2023):
            y = str(year)
            ym1= str(year+1)
            seleccionlink['Selección'].append(seleccion[l])
            seleccionlink['Link'].append('https://www.transfermarkt.mx' + linksolo[l] + '?saison_id=' + y)
            numeros=re.findall(r'\d+', linksolo[l])
            idsele=''.join(numeros)
            seleccionlink['Inicio'].append('01.01.'+ ym1)
            seleccionlink['Fin'].append('31.12.'+ ym1)
            seleccionlink['Inicio (T-1)'].append('01.01.'+ y)
            seleccionlink['Fin (T-1)'].append('31.12.'+ y)
            seleccionlink['IdSelección'].append(idsele)
            seleccionlink['Year'].append(y)

In [ ]:

## El siguiente código utiliza lo que obtuvimos en la celda anterior para obtener el nombre, nombre completo, minutos jugados 
## con el equipo en el año convocado y los minutos jugados en el año anterior. Al final, guarda en un diccionario todos estos 
## datos y agrega el equipo nacional, el año convocado y el año del que son los minutos.

minutosensele={'Name':[],'UniqueName':[],'NationalTeam':[],'YearSum':[],'YearMins':[],'MinutesNT':[],'MinutesNT(T-1)':[]}

for i in range(0,n):## este número se modifica de acuerdo al equipo y año de la que queremos que obtenga los datos
    selección = seleccionlink['Selección'][i]
    inicio = seleccionlink['Inicio'][i]
    fin = seleccionlink['Fin'][i]
    iniciotm1 = seleccionlink['Inicio (T-1)'][i]
    fintm1 = seleccionlink['Fin (T-1)'][i]
    añoconv = int(seleccionlink['Year'][i])+1
    añomins = int(seleccionlink['Year'][i])
    idsele = seleccionlink['IdSelección'][i]
    page = seleccionlink['Link'][i]
    headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
    page_tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(page_tree.content, 'html.parser')
    table= soup.find('table', {'class':'items'})
    table_body= table.find('tbody')
    tds= table_body.find_all('td', {'class':'zentriert'})
    inline_tables=table_body.find_all('table', {'class':'inline-table'})
    nameypos={'Name':[],'Links':[],'LinksrendSele':[], 'LinksrendSele (T-1)':[], 'NombreID':[]}
    for x in inline_tables:
        player = x.find('img')
        nameypos['Name'].append(player['alt'])
    getlink=[]
    for row in inline_tables:
        player_link = row.find('td', {'class':'hauptlink'})
        getlink.append(player_link)
    profile=[]
    for l in getlink:
        href=l.find('a')
        profile.append(href['href'])
    for h in profile:
        if h is not None:
            nameypos['Links'].append('https://www.transfermarkt.mx'+ h)
        else:
            nameypos['Links'].append('No Link')
    linksperfilselec=[]
    for link in nameypos['Links']:
        perfil=link
        rend= perfil.replace('profil', 'nationalmannschaft')
        linksperfilselec.append(rend)
    for k in range(0, len(linksperfilselec)):
        nameypos['LinksrendSele'].append(linksperfilselec[k]+'/verein_id/'+idsele+'/plus/0?hauptwettbewerb=&wettbewerb_id=&trainer_id=&start='+inicio+'&ende='+fin+'&nurEinsatz=1')
        nameypos['LinksrendSele (T-1)'].append(linksperfilselec[k]+'/verein_id/'+idsele+'/plus/0?hauptwettbewerb=&wettbewerb_id=&trainer_id=&start='+iniciotm1+'&ende='+fintm1+'&nurEinsatz=1')
    for i in range(0, len(nameypos['Name'])):
        page = nameypos['Links'][i]
        headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
        page_tree = requests.get(page, headers=headers)
        soup = BeautifulSoup(page_tree.content, 'html.parser')
        info=soup.find('div', {'class':'data-header__details'})
        items=info.find_all('ul')
        fichatec=soup.find('div',{'class':'info-table info-table--right-space'})
        if fichatec is not None:
            span=fichatec.find_all('span',{'class':'info-table__content info-table__content--bold'})
            nombrecomp=span[0].text
            nameypos['NombreID'].append(nombrecomp)
        else:
            nameypos['NombreID'].append('NA')
    for i in range(0, len(nameypos['Name'])):
        page= nameypos['LinksrendSele'][i]
        headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
        page_tree = requests.get(page, headers=headers)
        soup = BeautifulSoup(page_tree.content, 'html.parser')
        box=soup.find_all('div',{'class':'responsive-table'})
        if box is not None:
            if len(box)>1:
                spec_box=box[1].find('tbody')
                rows=spec_box.find_all('td',{'class':'rechts'})
                minutos=[]
                for minu in rows:
                    m=minu.text
                    numeros=re.findall(r'\d+',m)
                    minutes= ''.join(numeros)
                    if minutes=='':
                        minutos.append(int('0'))
                    else:
                        minutos.append(int(minutes))
                minutosensele['MinutesNT'].append(sum(minutos))
            else:
                minutosensele['MinutesNT'].append(int('0'))
        else: 
            minutosensele['MinutesNT'].append(int('0'))
            
    for i in range(0, len(nameypos['Name'])):
        nombreid=nameypos['NombreID'][i]
        nombre=nameypos['Name'][i]
        page= nameypos['LinksrendSele (T-1)'][i]
        headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
        page_tree = requests.get(page, headers=headers)
        soup = BeautifulSoup(page_tree.content, 'html.parser')
        box=soup.find_all('div',{'class':'responsive-table'})
        if box is not None:
            if len(box)>1:
                spec_box=box[1].find('tbody')
                rows=spec_box.find_all('td',{'class':'rechts'})
                minutos=[]
                for minu in rows:
                    m=minu.text
                    numeros=re.findall(r'\d+',m)
                    minutes= ''.join(numeros)
                    if minutes=='':
                        minutos.append(int('0'))
                    else:
                        minutos.append(int(minutes))
                minutosensele['MinutesNT(T-1)'].append(sum(minutos))
                minutosensele['Name'].append(nombre)
                minutosensele['UniqueName'].append(nombreid)
                minutosensele['NationalTeam'].append(selección)
                minutosensele['YearSum'].append(añoconv)
                minutosensele['YearMins'].append(añomins)
            else:
                minutosensele['MinutesNT(T-1)'].append(int('0'))
                minutosensele['Name'].append(nombre)
                minutosensele['UniqueName'].append(nombreid)
                minutosensele['NationalTeam'].append(selección)
                minutosensele['YearSum'].append(añoconv)
                minutosensele['YearMins'].append(añomins)
        else: 
            minutosensele['MinutesNT(T-1)'].append(int('0'))
            minutosensele['Name'].append(nombre)
            minutosensele['UniqueName'].append(nombreid)
            minutosensele['NationalTeam'].append(selección)
            minutosensele['YearSum'].append(añoconv)
            minutosensele['YearMins'].append(añomins)

In [ ]:
minutos=pd.DataFrame(minutosensele)

In [ ]:
minutos.to_excel('Nombre.xlsx', encoding='utf')